# Pipelines - Automating data preprocessing



In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('ggplot')
import pandas as pd
import numpy as np

# Display all cell outputs
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

from IPython import get_ipython
ipython = get_ipython()

%matplotlib inline
%config InlineBackend.figure_format = 'svg'

# Set max rows and columns displayed in jupyter
pd.set_option("display.max_rows", 100)
pd.set_option("display.max_columns", 100)

#the following gives access to utils folder
#where utils package stores shared code
import os
import sys
PROJECT_ROOT = os.path.abspath(os.path.join(
                  os.getcwd(),
                  os.pardir)
)

#only add it once
if (PROJECT_ROOT not in sys.path):
    sys.path.append(PROJECT_ROOT)
    
# autoreload extension
if 'autoreload' not in ipython.extension_manager.loaded:
    %load_ext autoreload

%autoreload 2

## Load our t-shirts


In [2]:
import utils as ut

#load raw t-shirt order
df = ut.generate_tshirt_order(100,100,100,dups=100, percent_nans=0.2)
df.iloc[1,3]='"-,.."'
df.head()

,weight,t_shirt_size,t_shirt_color,name,Age
0,92.510151,small,black,Betty Lindauer,8
1,115.379154,small,red,"""-,..""",8
2,109.617418,small,black,Amanda Costa,9
3,78.743580,small,blue,Robert Wilcoxen,12
4,86.019176,NaN,green,Robert Dense,14


# Transforms

Here is a suggested list of steps to automate data pre processing, given in the order they should be applied.  

1. Impute NaNs (or delete if there are not too many) --have to decide on strategy
2. Process strings
3. Delete duplicates  -- have to decide which fields to consider
4. Determine categorical columns
   a. convert ordinal categorical columns to numeric (may need to generate a dict for this)
   b. convert nominal categorical columns to one hot encoded columns
5. At this point all relevant data is numeric
6. Drop no variance columns (df.col.nunique returns 1)
7. Scale data (use standardization)
8. Feature Reduction-Dump correlated columns
9. Feature Reduction-apply PCA

This list is not complete, nor exact; for instance you might have the following columns in your dataset.

![](./43_pipeline_complete_preprocess_img2.png)

LocationDesc is a nominal categorical variable, 1 hot encoding it will add 49 additional columns to Dataset.  GeoLocation, OTOH, can be split into just 2 columns which provide both state  AND proximity to other states information. SO drop LocationDesc, keep and convert GeoLocation.

## Import Transforms that live in ../utils.transforms folder

If you want, or just use the ut prefix to get to them after you import utils as ut

In [3]:
#if you import like this you can use the function names directly
#without the ut. prefix
# from utils.transforms import *

## Apply transforms


In [4]:
# this is the hand coded bit for ordinal cat var
vals ={'t_shirt_size': {'large': 2, 'medium': 1, 'small': 0}}

In [5]:
#save these to operate on, can just as easily operate on original df
dfs=df.copy()
dfp=df.copy()
dfp2=df.copy()

### Either sequentially

if you import the functions via from utils.transforms import *

In [6]:
dfs=impute_NaNs(dfs)
dfs=ps_lower_strip(dfs,['name'])
dfs=ps_replace_punctuation(dfs,['name'])
dfs=remove_duplicates(dfs,['name'])

# is the hand coded bit for nominal cat var
vals ={'t_shirt_size': {'large': 2, 'medium': 1, 'small': 0}}
dfs=cat_ordinal(dfs, ['t_shirt_size'], vals)
dfs=drop_no_variance_columns(dfs)
dfs=scale(dfs)   #put this here if you do not want to scale the dummies
dfs=cat_getdummies(dfs, ['t_shirt_color'])
dfs=drop_correlated_columns(dfs)
dfs.head()

NameError: name 'impute_NaNs' is not defined

### Or as a pipeline (functionally equivelent to above). A pipeline is just a way to take the output of 1 function and feed it into another multiple times.  Just like above but in a 1 liner

In [ ]:
import utils as ut
#run a pipeline of transforms, note all functions are from ut namespace
dfp2=dfp2.pipe(ut.impute_NaNs).pipe(ut.ps_lower_strip).pipe(ut.ps_replace_punctuation,['name']).pipe(ut.remove_duplicates,['name']).pipe(ut.cat_ordinal,['t_shirt_size'],vals).pipe(ut.drop_no_variance_columns).pipe(ut.scale).pipe(ut.cat_getdummies, ['t_shirt_color']).pipe(ut.drop_correlated_columns)
dfp2.head()

Fixing 63 NaNs using most_frequent strategy
Removing 14 duplicate rows


,weight,t_shirt_size,name,Age,t_shirt_color_black,t_shirt_color_blue,t_shirt_color_green,t_shirt_color_orange,t_shirt_color_red
0,-0.940822,-0.928302,larry elias,-0.122604,False,False,False,True,False
1,-1.142119,-0.928302,,-0.846211,False,False,False,False,True
2,-0.807143,-0.928302,morgan tartaglia,-0.484408,False,True,False,False,False
3,-0.454465,-0.928302,william missey,-0.484408,False,False,True,False,False
4,-1.795704,-0.928302,don pratt,0.962806,False,False,False,True,False


In [ ]:
dfp.info()

#notice that everything is numerical (except for name), name is not very useful for a ML algorithm
dfp.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 315 entries, 0 to 314
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   weight         315 non-null    float64
 1   t_shirt_size   252 non-null    object 
 2   t_shirt_color  315 non-null    object 
 3   name           315 non-null    object 
 4   Age            315 non-null    int64  
dtypes: float64(1), int64(1), object(3)
memory usage: 12.4+ KB


,weight,Age
count,315.000000,315.000000
mean,138.553396,12.342857
std,39.572148,2.758295
min,53.493578,8.000000
25%,105.977106,10.000000
50%,131.758890,12.000000
75%,169.175419,15.000000
max,238.612934,17.000000


## Save to feather

In [ ]:
dfs.to_feather('preprocess.feather')

In [ ]:
dfs = pd.read_feather('preprocess.feather')
dfs

,weight,t_shirt_size,name,Age,t_shirt_color_black,t_shirt_color_blue,t_shirt_color_green,t_shirt_color_orange,t_shirt_color_red
0,-1.293759,-1.416874,geraldine schutz,-0.603395,0,0,0,0,1
1,-1.114754,-0.018582,,-1.657294,1,0,0,0,0
2,-1.357024,-0.018582,richard knox,-0.954695,0,0,1,0,0
3,-1.272972,-1.416874,alan peterson,-0.252095,0,0,0,1,0
4,-0.559187,-1.416874,linda robinson,-1.305994,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...
296,-0.454675,1.379710,cheryl ochoa,0.450504,0,0,0,0,1
297,-0.031135,1.379710,wilbert thomas,0.450504,0,1,0,0,0
298,0.400668,1.379710,ruth rybak,-0.603395,1,0,0,0,0
299,0.507852,1.379710,kimberly verity,1.153103,0,0,0,0,1
